# Задача 2

Написать код для решения задачи оптимизации параметров промежуточного перегрева P_пп и t_пп для свободных начальных параметров p_0,t_0, p_к. Сделаем допущение, что начальная точка процесса расширения всегда находится в зоне перегретого пара. Решение должно выдавать параметры промежуточного перегрева и термический КПД при них.

In [1]:
import iapws
import array
from typing import Optional, Tuple, List, Union
from iapws import IAPWS97 as gas
from scipy.optimize import minimize
import numpy as np
point_type = iapws.iapws97.IAPWS97

In [2]:
##Упростим себе жизнь
MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None


In [11]:
def efficiency(middle_point):
    p_middle = middle_point[0]
    t_middle = middle_point[1]
    point_0_start = gas(P=p_0 * unit, T=t_0)
    point_0_end = gas(P=p_middle * unit, s=point_0_start.s)
    point_1_start = gas(P=p_middle * unit, T=t_middle)
    point_condenser = gas(P=p_k * unit, s=point_1_start.s)
    point_pump = gas(P=p_k * unit, x=0)
    useful_energy = (point_0_start.h - point_0_end.h) + (point_1_start.h - point_condenser.h)
    full_energy = (point_0_start.h - point_pump.h) + (point_1_start.h - point_0_end.h)
    efficiency = useful_energy / full_energy
    return 1 - efficiency
    


In [12]:
def point(p_0,t_0,p_k):
    middle_point = np.array([p_0 * 0.7, t_0 * 0.7 ])
    bound = [(1, p_0), (1, t_0)]
    rezul = (minimize(efficiency,middle_point,bounds=bound).x)
    return rezul

In [26]:
p_0 = float(input("Введите p_0 "))
p_0 = p_0 * MPa
t_0 = float(input("Введите t_0 ")) 
p_k = float(input("Введите p_k "))
p_k = p_k * kPa
rez = (point(p_0,t_0,p_k))
p_middle = round(rez[0] * unit,3)
t_middle = rez[1]
print("Давление промперегрева",p_middle,"MPa")
print("Температура промперегрева",t_middle,"К")


Введите p_0 10
Введите t_0 500
Введите p_k 5
Давление промперегрева 7.0 MPa
Температура промперегрева 500.0 К
